In [8]:
using Distributed
using DifferentialEquations
using Plots

addprocs(exeflags="--project=$(Base.active_project())")
@everywhere using DifferentialEquations


In [9]:
# Linear ODE which starts at 0.5 and solves from t=0.0 to t=1.0
prob = ODEProblem((u, p, t) -> 1.01u, 0.5, (0.0, 1.0)) # 解くべきODEが"(u, p, t) -> 1.01u”, 初期条件u0=0.5, tsapn=(0.0,1.0)

ODEProblem with uType Float64 and tType Float64. In-place: false
timespan: (0.0, 1.0)
u0: 0.5

In [10]:
@everywhere function prob_func(prob, i, repeat)
    remake(prob, u0=rand() * prob.u0)
end

In [16]:
ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, Tsit5(), EnsembleDistributed(), trajectories=100)

size(sim,2)

100

In [12]:
plotly()
plot(sim, linealpha=0.4)

In [13]:
using DifferentialEquations
prob = ODEProblem((u, p, t) -> 1.01u, 0.5, (0.0, 1.0))
function prob_func(prob, i, repeat)
    remake(prob, u0=rand() * prob.u0)
end
ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, Tsit5(), EnsembleThreads(), trajectories=10)

sim[1]

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 5-element Vector{Float64}:
 0.0
 0.0996570929793885
 0.3457510213075278
 0.6777806714963273
 1.0
u: 5-element Vector{Float64}:
 0.3663586758702287
 0.4051537019804564
 0.519476395419094
 0.7264511877528333
 1.005874703546196

In [3]:
using LinearAlgebra

# Definition of Equation of Motion of Planar CRTBP
function eom_pcrtbp!(dx, x, μ, t)
    # Preparation
    r1 = x[1:2] + [μ, 0.0]
    r2 = x[1:2] - [1.0 - μ, 0.0]
    a_cori = [2.0x[4], -2.0x[3]]
    a_cf = [x[1], x[2]]
    # dx/dt
    dx[1:2] = x[3:4]
    dx[3:4] = -(1.0 - μ) * r1 / (norm(r1)^3) - μ * r2 / (norm(r2)^3) + a_cori + a_cf
end

eom_pcrtbp! (generic function with 1 method)

In [4]:
using DifferentialEquations

# ODEの条件・問題設定（後でアンサンブル・シミュレーションを行う際に値を変更できるので，ひとまず仮置き）
μ = 0.01215058426994 # 地球・月系を仮定
x0 = [0.8369304414031459, -4.265566441114701, -4.076205405804159, -0.43142925920758124]
tspan = (0.0, 3.0pi)
prob = ODEProblem(eom_pcrtbp!, x0, tspan, μ)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 9.42477796076938)
u0: 4-element Vector{Float64}:
  0.8369304414031459
 -4.265566441114701
 -4.076205405804159
 -0.43142925920758124

In [5]:
function prob_func(prob, i, repeat)
    x0 = [rand(), -4.265566441114701, -4.076205405804159, -0.43142925920758124]
    remake(prob, u0=x0)
end

prob_func (generic function with 1 method)

In [6]:
ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, Vern7(), EnsembleDistributed(), trajectories=10)

EnsembleSolution Solution of length 10 with uType:
ODESolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{Float64}}}, ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, Float64, ODEFunction{true, SciMLBase.AutoSpecialize, FunctionWrappersWrappers.FunctionWrappersWrapper{Tuple{FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{Float64}, Vector{Float64}, Float64, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, Float64, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, Vector{Float64}, Float64, ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vec